In [1]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install pathlib

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python38\python.exe -m pip install --upgrade pip' command.


In [2]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch
import pandas as pd
from pathlib import Path

In [5]:
if torch.cuda.is_available():
  print("\nUsing: ", torch.cuda.get_device_name(0))
  device = torch.device('cuda')
else:
  print("\nUsing: CPU")
  device = torch.device('cpu')


Using:  GeForce GTX 1070


In [6]:
#Load trained model
model_dir = "model"
model = BertForSequenceClassification.from_pretrained(model_dir, num_labels=3)
tokenizer = BertTokenizer.from_pretrained(model_dir)
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [7]:
def load_dataset (data_filename):
  data_path = Path(data_filename)
  if data_path.exists():
    print(f"Loading {data_path}...")
    data_csv = pd.read_csv(data_path)
    return data_csv
  else:
    print(f"The dataset in {data_path} does not exist")

In [8]:
#Load dataset
data = load_dataset("raw_partner_headlines.csv")

Loading raw_partner_headlines.csv...


In [78]:
def get_pred(sentence):
  encoded_sentence = tokenizer.encode_plus(
        sentence,                      # input the news headline
        add_special_tokens = True,     # special tokens added to mark beginning & end of sentence
        truncation = True,             # make all sentences a fixed length
        padding = 'max_length',        # pad with zeros to max length
        return_attention_mask = True,  # include attention mask in encoding
        return_tensors = 'pt'          # return as pytorch tensor
    )
  input_ids = encoded_sentence['input_ids'].to(device)
  attention_mask = encoded_sentence['attention_mask'].to(device)
  out = model(input_ids, attention_mask)
  pred = torch.argmax(out[0])
  return pred.item()

In [79]:
sentiments = []
for index, row in data.iterrows():
  sentiments.append(get_pred(row['headline']))
data['sentiment'] = sentiments
print(data)
data.to_csv("stock_sentiment_data.csv", index=False, header = True, encoding='utf-8-sig')

         Unnamed: 0                                           headline  \
0                 2  Agilent Technologies Announces Pricing of $5……...   
1                 3  Agilent (A) Gears Up for Q2 Earnings: What's i...   
2                 4  J.P. Morgan Asset Management Announces Liquida...   
3                 5  Pershing Square Capital Management, L.P. Buys ...   
4                 6  Agilent Awards Trilogy Sciences with a Golden ...   
...             ...                                                ...   
1845554     1849874                      Consumer Cyclical Sector Wrap   
1845555     1849875                      Consumer Cyclical Sector Wrap   
1845556     1849876  Zacks #5 Rank Additions for Monday - Tale of t...   
1845557     1849877  4 Stock Strategies From Wall Street: Feb. 9 (U...   
1845558     1849878        4 Stock Strategies From Wall Street: Feb. 9   

                                                       url      publisher  \
0        http://www.gurufocus.com/